# Getting started with the XTRF HP API

Pre-Requirements:
* Python 3 including the library Requests 
* Jupyter Lab
* A valid HP API token for the XTRF instance you are working with

TODO:

1. create a classic project
2. upload a file
3. add this file to one of the tasks


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>


## 1. Create a classic project

First, we import three libraries to help us deal with HTTP requests, JSON responses and epoch time. At the same time, we also import some information that is user specific from the file secrets, namely the server base URL and the HP API token that will be used to authenticate our calls to the server.



In [1]:
import requests
import json
import time

from secrets import server_url, token  # edit the secrets.py file and enter your details

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In the API documentation we find out that a POST request to the endpoint "/projects" will create a new Classic Project.

![img](images/post_projects.png)
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
There are 5 HTTP request methods:
* GET (read data)
* POST (send data)
* PUT (send data to replace/update server side data)
* DELETE (delete server side data)
* (PATCH) (update data, but rarely used) 
<br><br>

Our POST HTTP request will send data to the server.


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
The "/projects" POST endpoint takes data in JSON format, structured in a specific way. (This is the information XTRF uses to create the project.)


In [ ]:
project_model = {
          "customerId": 1,                      # <- This ID is specific to the XTRF instance / REQUIRED
          "sourceLanguageId": 56,               # <- This ID is specific to the XTRF instance 
          "targetLanguagesIds": [               # <- This square bracket starts an "array" 
                                                # if no TL ids are specified, the project will be created without tasks
            90,                                 # <- This ID is specific to the XTRF instance
            101,                                # <- This ID is specific to the XTRF instance
            49                                  # <- This ID is specific to the XTRF instance
          ],                                    # <- This square bracket closes the array
          "specializationId": 7,                # <- This ID is specific to the XTRF instance / REQUIRED
          "serviceId": 8,                       # <- This ID is specific to the XTRF instance / REQUIRED
          "dates": {                            # <- This curly brace opens a nested JSON object
            "startDate": {                      # <- optional, project will get time of creating if not specified
              "time": 1606465852000             # <- so-called epoch time, in milliseconds
            },
            "deadline": {                       # <- optional, project can be created w/o specifying DL
              "time": 1606638403369             # <- so-called epoch time, in milliseconds
            }
          },                                    # <- This curly brace closes the nested JSON object
          "name": "The API generated project",  # <- The name of the project, will be carried over to tasks (optional)
          "people": {                           # <- people is optional, XTRF will choose defaults if not specified
            "responsiblePersons": {
              "projectManagerId": 1             # <- This ID is specific to the XTRF instance 
            },
            "customerContacts": {               # <- customerContacts is optional, if not set, XTRF will add default customer contact
                "primaryId": 1,                 # <- This ID is specific to the XTRF instance
                "sendBackToId": 2               # <- This ID is specific to the XTRF instance
            }
          },
          "instructions": {                     # <- instructions is optional
            "forProvider": "You don't have to put instructions to the vendors here, it's optional. \
                            But if you do, they end up at the project level",
            "fromCustomer": "You don't have to put instructions to the customer here either, it's also optional. \
                            But if you do, they also end up at the project level",
          },
          "categoriesIds": [                    # <- categoriesIds is optional
            5,
            1
          ],
          "inputFiles": [                       # <- inputFiles is optional too, and takes an array of file models
            {                                   # <- the file model must contain one of the following: url, token or content.
                "name": "openapi.json",
                "url": "https://summithenrik.s.xtrf.eu/home-api/openapi.json",
                "token": "string_returned_when_uploading_a_file",
                "content": "This string will be converted to an RTF file, just like when you add a string in the UI",
                "category": "WORKFILE"          # One of the following: WORKFILE, TM, DICTIONARY, REF, LOG_FILE
              
            }
          ]
        }


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

Things get easier if you think of this as a structured way of bundling all the clicks and inputs you would normally perform when registering a project using the XTRF home portal UI. 

![img](images/1.png) 
<br><br><br><br><br><br><br>
![img](images/2.png)



<br><br><br><br><br><br><br><br>








<br><br><br><br><br><br>
#### Dates

For Unix based systems, the current epoch started January 1 1970, 00:00 and epoch time of some point in time is the number of seconds between epoch start and that point in time.
<br><br><br><br><br><br>

In [7]:
now = time.time()  # time since epoch start in seconds
now_in_ms = now * 1000  # time since epoch start in milliseconds
two_days_in_ms = 60 * 60 * 24 * 2 * 1000 #  60 seconds * 60 minutes * 24 hours * 2 = 48 hours from now

deadline = now_in_ms + two_days_in_ms # now in milliseconds + 48 hours worth of milliseconds

In [8]:
time.time() * 1000

1607501253173.5652

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

### Preparing the request

We know the endpoint, we know the frame data of the project we want to create. Last thing we need is to authentication, so let's that up and prepare the request. 

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [9]:
# authentication header - this will remain the same and be used in all subsequent requests - the token was imported right at the start

auth_header = {"X-AUTH-ACCESS-TOKEN": token}  # from the docs: "Access token must be provided in a static HTTP header "X-AUTH-ACCESS-TOKEN" 

In [11]:
#  endpoint - base URL is https://YOUR_XTRF_URL/home-api
url = server_url + "/projects"
url

'https://summithenrik.s.xtrf.eu/home-api/projects'

In [12]:
#  data 
data = {
          "customerId": 1,                      # <- This ID is specific to the XTRF instance 
          "sourceLanguageId": 219,              # <- This ID is specific to the XTRF instance 
          "targetLanguagesIds": [               # <- This square bracket starts an array  
            73                                  # <- This ID is specific to the XTRF instance 
          ],                                    # <- This square bracket closes the array
          "specializationId": 7,                # <- This ID is specific to the XTRF instance
          "serviceId": 8,                       # <- This ID is specific to the XTRF instance
          "dates": {                            # <- This curly brace opens a nested JSON object
            "startDate": {
              "time": now_in_ms                 # <- so-called epoch time, in milliseconds - set above
            },
            "deadline": {
              "time": deadline                  # <- so-called epoch time, in milliseconds - also set above
            }
          },                                    # <- This curly brace closes the nested JSON object
          "name": "Commented project creation",             # <- The name of the project, will be carried over to tasks 
          "people": {
            "responsiblePersons": {
              "projectManagerId": 1             # <- This ID is specific to the XTRF instance 
            },
            "customerContacts": {               # <- customerContacts is optional, if not set, XTRF will add default customer contact
                "primaryId": 1,                 # <- This ID is specific to the XTRF instance
                "sendBackToId": 2               # <- This ID is specific to the XTRF instance
            }
          },
          "instructions": {                     # <- instructions are optional
            "forProvider": "Please translate this!",
            "fromCustomer": "The naming of this field has always puzzled me. :)",
          }
        }

<br><br><br><br><br><br><br><br><br><br><br><br><br><br>
Now we are all set! The syntax for requests is straightforward:


In [13]:
response = requests.post(url, json=data, headers=auth_header)  # Requests has excellent docs here: https://requests.readthedocs.io/













In [ ]:
print(response.content)

In [ ]:
from pprint import pprint  # pprint makes prints in a way that is easier for humans to read
loaded_json = json.loads(response.content)

pprint(loaded_json)

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
Here we see that the POST to create the project does not return the full project data - the tasks are missing!

Therefore we need to make another request to get the complete picture, and there is an endpoint for this:

![img](images/get_projects_projectid.png)

Here we see two things: 
1. To get the project data, we need to reference the project ID in the endpoint. Good thing we saved the response!
2. To get the task data, we need to add a parameter to the call. 

So let's set up that request!

In [ ]:
# project ID
project_id = str(loaded_json["id"])
project_id

In [ ]:
# endpoint
url = server_url + "/projects/" + project_id
url

In [ ]:
#parameters
parameters = {"embed": "tasks"}  # see the Requests docs about passing parameters: https://requests.readthedocs.io/en/master/user/quickstart/#passing-parameters-in-urls

In [ ]:

response = requests.get(url, params=parameters, headers=auth_header)
complete_project_data = json.loads(response.content)

In [ ]:
pprint(complete_project_data)

# 2. Uploading a file

Uploading files to XTRF is a two-step process: First you send the file data to XTRF, which then responds with token, which will be used to refer to that file in a subsequent call. 

![img](images/post_files.png)

Setting up the request, we need to prepare the file data by opening the file and reading its contents and prepare the meta-data that must be sent along with the file data:

In [ ]:
#  endpoint
url = server_url + "/files"

#  file contents
file = {'file': open("source_file.docx", "rb")}

#  file meta data (name)
data = {"fileName": "source_file.docx"}
            


In [ ]:
response = requests.post(server_url + "/files", files=file, json=data, headers=auth_header)

In [ ]:
loaded_json = json.loads(response.content)
pprint(loaded_json)

In [ ]:
file_token = loaded_json["token"]


So, we have uploaded the file, and have the token to prove it. Onwards to the last step:

## 3. Adding the file to a task

We now have the project data, including task IDs. And we have uploaded the source file to XTRF. Last thing we shall do is to add it to one of the tasks. 

Let's look at the documentation.

![img](images/post_task_files_input.png)

Here we will do another POST, and we need to know the task ID + file token and decide on what file category this is.

Setting up the request:

In [ ]:
#  task ID 
task_id = str(complete_project_data["tasks"][0]["id"])  # we get the task ID from the complete project data from step 1


#  endpoint
url = server_url + "/tasks/" + task_id + "/files/input"

#  data
data = {"token": file_token, 
        "category": "WORKFILE"}

In [ ]:
final_response = requests.post(url, json=data, headers=auth_header)

In [ ]:
final_response.status_code
